In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
)
from torch.utils.data import DataLoader
from evaluate import load
import os
import numpy as np
import json
from PIL import Image

In [2]:
# === SETUP ===
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device found")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [3]:
# === PATH TO DATASET ===
data_dir = "New Plant Diseases Dataset"  

In [4]:
# === PROCESSOR ===
processor = AutoImageProcessor.from_pretrained("google/efficientnet-b2")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
# === TRANSFORMS ===
transform = transforms.Compose([
    transforms.Resize((260, 260)),  # Required size for efficientnet-b2
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

In [7]:
# === LOAD DATA ===
train_dataset = ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset = ImageFolder(os.path.join(data_dir, "valid"), transform=transform)

In [8]:
# ✅ CLASS NAME MAPPING
class_names = train_dataset.classes
label_map = {i: name for i, name in enumerate(class_names)}
num_labels = len(class_names)
print("✅ Total classes:", num_labels)
print("✅ Label map:", label_map)

✅ Total classes: 38
✅ Label map: {0: 'Apple___Apple_scab', 1: 'Apple___Black_rot', 2: 'Apple___Cedar_apple_rust', 3: 'Apple___healthy', 4: 'Blueberry___healthy', 5: 'Cherry_(including_sour)___Powdery_mildew', 6: 'Cherry_(including_sour)___healthy', 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn_(maize)___Common_rust_', 9: 'Corn_(maize)___Northern_Leaf_Blight', 10: 'Corn_(maize)___healthy', 11: 'Grape___Black_rot', 12: 'Grape___Esca_(Black_Measles)', 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 14: 'Grape___healthy', 15: 'Orange___Haunglongbing_(Citrus_greening)', 16: 'Peach___Bacterial_spot', 17: 'Peach___healthy', 18: 'Pepper,_bell___Bacterial_spot', 19: 'Pepper,_bell___healthy', 20: 'Potato___Early_blight', 21: 'Potato___Late_blight', 22: 'Potato___healthy', 23: 'Raspberry___healthy', 24: 'Soybean___healthy', 25: 'Squash___Powdery_mildew', 26: 'Strawberry___Leaf_scorch', 27: 'Strawberry___healthy', 28: 'Tomato___Bacterial_spot', 29: 'Tomato___Early_blight', 30

In [9]:
# === SAVE LABEL MAP ===
output_dir = "efficientnet-plant-disease"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "label_map.json"), "w") as f:
    json.dump(label_map, f)

In [10]:
# === LOAD MODEL ===
model = AutoModelForImageClassification.from_pretrained(
    "google/efficientnet-b2",
    num_labels=num_labels,
    ignore_mismatched_sizes=True
).to(device)

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1408]) in the checkpoint and torch.Size([38, 1408]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([38]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# === COLLATE FUNCTION ===
def collate_fn(batch):
    pixel_values = torch.stack([x[0] for x in batch])
    labels = torch.tensor([x[1] for x in batch])
    return {"pixel_values": pixel_values, "labels": labels}

In [12]:
# === METRICS ===
accuracy = load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

In [13]:
# === TRAINING ARGS ===
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    remove_unused_columns=False,
)

In [14]:
# === TRAINER ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=None,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

C:\Users\bhara\AppData\Local\Temp\ipykernel_9912\2092077056.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# === TRAIN ===
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.041700,0.160219,0.958115
2,0.026800,0.053440,0.985318
3,0.024800,0.120212,0.968245
4,0.001500,0.013670,0.997155
5,0.008800,0.025053,0.994195


TrainOutput(global_step=21970, training_loss=0.07902896913575708, metrics={'train_runtime': 5484.4849, 'train_samples_per_second': 64.085, 'train_steps_per_second': 4.006, 'total_flos': 3.31641947780208e+18, 'train_loss': 0.07902896913575708, 'epoch': 5.0})

In [16]:
# === SAVE MODEL + PROCESSOR ===
trainer.save_model(output_dir)
processor.save_pretrained(output_dir)
print(f"✅ Model + processor saved to `{output_dir}/`")

✅ Model + processor saved to `efficientnet-plant-disease/`
